In [1]:
import PuzzleTools
using BenchmarkTools

In [2]:
import Collective

In [3]:
words = PuzzleTools.Wordsets.unixwords()[1:10000:end];

In [4]:
reload("Collective")

In [4]:
@benchmark Collective.Corpus(words)

BenchmarkTools.Trial: 
  memory estimate:  3.09 mb
  allocs estimate:  95986
  --------------
  minimum time:     64.656 ms (0.00% GC)
  median time:      71.036 ms (0.00% GC)
  mean time:        71.428 ms (0.65% GC)
  maximum time:     81.322 ms (0.00% GC)
  --------------
  samples:          70
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [14]:
c = Collective.Corpus(words)

Collective.Corpus(Collective.Feature[Collective.ContainsLetter('a'),Collective.ContainsLetter('b'),Collective.ContainsLetter('c'),Collective.ContainsLetter('d'),Collective.ContainsLetter('e'),Collective.ContainsLetter('f'),Collective.ContainsLetter('g'),Collective.ContainsLetter('h'),Collective.ContainsLetter('i'),Collective.ContainsLetter('j')  …  Collective.NumUniqueLetters(1),Collective.NumUniqueLetters(2),Collective.NumUniqueLetters(3),Collective.NumUniqueLetters(4),Collective.NumUniqueLetters(5),Collective.NumUniqueLetters(6),Collective.NumUniqueLetters(7),Collective.NumUniqueLetters(8),Collective.NumUniqueLetters(9),Collective.NumUniqueLetters(10)],[0.615515,0.160237,0.365833,0.267062,0.672319,0.0962272,0.174226,0.234421,0.615515,0.0135651  …  0.000847817,0.00296736,0.0114455,0.0385757,0.0945316,0.151759,0.207291,0.172107,0.160237,0.103858])

In [15]:
score_dist = Dict{Int, Int}()
for word in words
    score = Collective.scrabble_score(word)
    score_dist[score] = get(score_dist, score, 0) + 1
end

In [16]:
scores = zeros(maximum(keys(score_dist)))
for (score, count) in score_dist
    scores[score] = count
end

In [17]:
F = cumsum(scores / sum(scores))

46-element Array{Float64,1}:
 0.000423908
 0.000423908
 0.000847817
 0.00593472 
 0.0131412  
 0.0258584  
 0.0517168  
 0.0796948  
 0.120814   
 0.175498   
 0.236965   
 0.303095   
 0.364561   
 ⋮          
 0.995761   
 0.997033   
 0.997457   
 0.998728   
 0.998728   
 0.999152   
 0.999152   
 0.999152   
 0.999576   
 0.999576   
 0.999576   
 1.0        

In [18]:
scores_exp = zeros(46)
scores_exp[14] = 5

5

In [19]:
Fn = cumsum(scores_exp / sum(scores_exp))

46-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [34]:
scores

46-element Array{Float64,1}:
   1.0
   0.0
   1.0
  12.0
  17.0
  30.0
  61.0
  66.0
  97.0
 129.0
 145.0
 156.0
 145.0
   ⋮  
   3.0
   3.0
   1.0
   3.0
   0.0
   1.0
   0.0
   0.0
   1.0
   0.0
   0.0
   1.0

In [37]:
scores_exp

46-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [92]:
?binomial

search: binomial BinomialTest



```
binomial(n,k)
```

Number of ways to choose `k` out of `n` items.


In [133]:
crossentropy(p, q) = -sum(p .* log.(q))

crossentropy (generic function with 1 method)

In [134]:
n = 7
p = scores / sum(scores)

x = zeros(Int, length(p))
for i in 1:n
    x[Collective.scrabble_score(rand(words))] += 1
#     x[14] += 1
end

slots = 0
for i in 1:length(x)
    if x[i] <= p[i] * n
        slots += x[i]
    else
        slots += (n - x[i])
    end
end
# correction = binomial(slots, n)
        
# factorial(n) / (prod(factorial(xi) for xi in x)) * prod(p[i]^x[i] for i in 1:length(p))
crossentropy(p, x)

NaN

In [130]:
MultinomialLRT([7], [p[14]])

LoadError: LoadError: Probabilities must sum to one
while loading In[130], in expression starting on line 1

In [112]:
slots, correction

(42,26978328)

In [76]:
pvalue(ApproximateTwoSampleKSTest(Collective.scrabble_score.(words), [14, 14, 14, 14, 14]))

0.08423702722876153

In [88]:
pvalue(KSampleADTest(Collective.scrabble_score.(words), 14 * ones(Int, 5)))

0.1042709289859092

In [23]:
pvalue(ApproximateTwoSampleKSTest(F, Fn))

4.297908655846245e-10

In [30]:
pvalue(KSampleADTest(vcat(falses(10), trues(10)), [true, true, true, true, true]))

0.006747097089861135

In [44]:
A = (Fn, F, n) -> n * sum((Fn[i] - F[i])^2 / (F[i] * (1 - F[i])) for i in 1:(length(F) - 1))

(::#11) (generic function with 1 method)

In [45]:
A(Fn, F, 5)

34.259516512945844

In [46]:
A([0, 1], [0.5, 1], 2)

2.0

In [131]:
d1s2 = ["leveraged", "sizes", "tuxedo", "lynx", "lightly", "crocodile", "triumph"]
sort(Collective.analyze(c, d1s2))[1:10]

10-element Array{Collective.FeatureResult,1}:
 Collective.FeatureResult(Collective.ScrabbleScore(14),Bool[true,true,true,true,true,true,true],1.00936e-8)               
 Collective.FeatureResult(Collective.LetterAtIndex('l',1),Bool[true,false,false,true,true,false,false],0.000598249)       
 Collective.FeatureResult(Collective.LetterAtIndex('z',3),Bool[false,true,false,false,false,false,false],0.00883437)      
 Collective.FeatureResult(Collective.ContainsLetter('a'),Bool[true,false,false,false,false,false,false],0.0139192)        
 Collective.FeatureResult(Collective.LetterAtIndexFromEnd('x',1),Bool[false,false,false,true,false,false,false],0.0146491)
 Collective.FeatureResult(Collective.ContainsLetter('x'),Bool[false,false,true,true,false,false,false],0.0154881)         
 Collective.FeatureResult(Collective.LetterAtIndexFromEnd('x',4),Bool[false,false,true,false,false,false,false],0.0175342)
 Collective.FeatureResult(Collective.AlternatesConsonantVowel(),Bool[true,true,true,false,fal

In [1]:
using HypothesisTests

INFO: Precompiling module HypothesisTests.


In [6]:
t = BinomialTest(5, 5, 0.1)
pvalue(t)

2.000000000000002e-5